In [45]:
import pandas as pd
import re

In [46]:
df = pd.read_csv('smallFebGMK.csv')
df.drop(df.columns[0], axis=1, inplace=True)
df.drop_duplicates(subset=["link"], inplace=True)
df.set_index(['author', 'link'], inplace=True)
#df.head()

In [47]:
df['post_lower'] = df['post'].str.lower()
df['title_lower'] = df['title'].str.lower()
only_gmk = df[df['post_lower'].str.contains('gmk') | df['title_lower'].str.contains('gmk')].copy()
#only_gmk.shape, df.shape

In [48]:
def split_on_gmk(x):
    a = x.lower().split("gmk")[1]
    return "gmk " + a.split()[0]

only_gmk['product_name'] = only_gmk['rawtext'].apply(split_on_gmk)
#only_gmk

In [49]:
print(only_gmk.index)
#only_gmk
#only_gmk.drop_duplicates(subset=["link"])

MultiIndex([(            'rath_k', ...),
            (       'Krasztest01', ...),
            ( 'Mr_BillButtlicker', ...),
            ( 'Mr_BillButtlicker', ...),
            ( 'Mr_BillButtlicker', ...),
            ('excited_to_be_here', ...),
            (             'Sedyn', ...),
            (    '_ProjectEuler_', ...),
            (    '_ProjectEuler_', ...),
            (          'Zeelobby', ...),
            ...
            (             'jsnau', ...),
            (             'fff_x', ...),
            (             'fff_x', ...),
            (        'Darkemajik', ...),
            (        'Darkemajik', ...),
            (        'Darkemajik', ...),
            (            'ra1nnn', ...),
            (             'Acabi', ...),
            (            'Clleno', ...),
            (            'Clleno', ...)],
           names=['author', 'link'], length=154)


In [64]:
from collections import defaultdict
all_prices = defaultdict(list)

money_regex = r'\$\d+|\d+\$'

sets = {'base':0, 'nov':0, 'alphas':0, 'accents':0, 'bars':0, 'light base':0, 'dark base':0, 'deskmat':0, 'rama':0}

def match_product(df):
    for row in df.itertuples():
        s = row.post_lower.split("\n")
        product = row.product_name.lower()
        for l in s:
            low = l.lower()
            if "gmk" in low and ('~~' in low or 'sold' in low):
                #print(low)
                a = low.split("gmk ")[1]
                product_name = "gmk " + re.split(r" |~", a)[0]
                #print(a, re.split(r" |~", a))
                matches = re.findall(money_regex, low)
                
                prices = [int(re.findall(r'\d+', m)[0]) for m in matches]
                if len(prices) > 0:
                    kits = []
                    removeBase = False
                    empty=True
                    for x in sets.keys():
                        if x in low:
                            empty=False
                            sets[x]+=1
                            if x == 'nov':
                                kits.append('novelties')
                            elif x == 'light base' or x == 'dark base':
                                kits.insert(0, f'{x}')
                                removeBase = True
                            else:
                                kits.append(x)
                    products = product_name
                    if not kits:
                        kits = ['base']
                    products+= f' {kits.pop(0)}'
                    for x in kits:
                        if removeBase and x == 'base':
                            continue
                        products += f', {x}'
                    print(f"PRODUCT: {products} LINE: {low} PRICES: {prices}")
                    all_prices[product_name].append(min(prices))
    

for i, new_df in only_gmk.groupby(level=0):
    match_product(new_df)

print(all_prices)

a alphas kit $110 shipped**~~ PRICES: [110]
PRODUCT: gmk relegendables base LINE: ~~**gmk relegendables $40 shipped**~~ sold local sale PRICES: [40]
PRODUCT: gmk camping deskmat LINE: |~~gmk camping deskmat~~|~~lightly used, in good condition~~|~~$30 shipped~~ sold for asking| PRICES: [30]
PRODUCT: gmk masterpiece rama LINE: ~~gmk masterpiece x rama master gold - $175~~ sold. PRICES: [175]
PRODUCT: gmk masterpiece rama LINE: ~~gmk masterpiece x rama knife black - $175~~ sold. PRICES: [175]
PRODUCT: gmk masterpiece rama LINE: ~~gmk masterpiece x rama knife gold - $175~~ sold. PRICES: [175]
PRODUCT: gmk nautilus rama LINE: ~~gmk nautilus 2 x rama nautilus enter - $200~~  PRICES: [200]
PRODUCT: gmk nautilus base LINE: - ~~sold: gmk nautilus nightmares - $200~~ PRICES: [200]
PRODUCT: gmk bingsu base LINE: # gmk bingsu - $350 shipped (sold) PRICES: [350]
PRODUCT: gmk dmg bars LINE: # gmk dmg spacebars - $45 shipped (sold) PRICES: [45]
PRODUCT: gmk olivia++ light base LINE: |~~gmk olivia++ l